In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
#notebook created by Dora Szabo on 22.11.2019 (https://github.com/szaboodoora)

Your goal is to predict the operating condition of a waterpoint for each record in the dataset. You are provided the following set of information about the waterpoints:

https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/25/

amount_tsh - Total static head (amount water available to waterpoint)
date_recorded - The date the row was entered
funder - Who funded the well
gps_height - Altitude of the well
installer - Organization that installed the well
longitude - GPS coordinate
latitude - GPS coordinate
wpt_name - Name of the waterpoint if there is one
num_private -
basin - Geographic water basin
subvillage - Geographic location
region - Geographic location
region_code - Geographic location (coded)
district_code - Geographic location (coded)
lga - Geographic location
ward - Geographic location
population - Population around the well
public_meeting - True/False
recorded_by - Group entering this row of data
scheme_management - Who operates the waterpoint
scheme_name - Who operates the waterpoint
permit - If the waterpoint is permitted
construction_year - Year the waterpoint was constructed
extraction_type - The kind of extraction the waterpoint uses
extraction_type_group - The kind of extraction the waterpoint uses
extraction_type_class - The kind of extraction the waterpoint uses
management - How the waterpoint is managed
management_group - How the waterpoint is managed
payment - What the water costs
payment_type - What the water costs
water_quality - The quality of the water
quality_group - The quality of the water
quantity - The quantity of water
quantity_group - The quantity of water
source - The source of the water
source_type - The source of the water
source_class - The source of the water
waterpoint_type - The kind of waterpoint
waterpoint_type_group - The kind of waterpoint

# Data cleaning and some EDA

In [2]:
train_features=pd.read_csv('C:/Users/dooora/Desktop/pumpitup/training_features.csv',index_col='id')
#C:\Users\dooora\Desktop\pumpitup
#C:/Users/dooora/Desktop/jupyter/pumpitup/training_features.csv
train_features.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [3]:
train_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 39 columns):
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
recorded_by              59400 non-null obj

In [4]:
train_labels=pd.read_csv('C:/Users/dooora/Desktop/pumpitup/training_labels.csv',index_col='id')
train_labels.head()

,status_group
id,
69572,functional
8776,functional
34310,functional
67743,non functional
19728,functional


In [5]:
test_features=pd.read_csv('C:/Users/dooora/Desktop/pumpitup/test_features.csv',index_col='id')
test_features.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [6]:
test_features.columns

Index(['amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer',
       'longitude', 'latitude', 'wpt_name', 'num_private', 'basin',
       'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward',
       'population', 'public_meeting', 'recorded_by', 'scheme_management',
       'scheme_name', 'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

In [7]:
train_features['train']=1
test_features['train']=0
combined = pd.concat([train_features, test_features])
combined.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,train
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,1
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,1
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1


In [8]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74250 entries, 69572 to 68707
Data columns (total 40 columns):
amount_tsh               74250 non-null float64
date_recorded            74250 non-null object
funder                   69746 non-null object
gps_height               74250 non-null int64
installer                69718 non-null object
longitude                74250 non-null float64
latitude                 74250 non-null float64
wpt_name                 74250 non-null object
num_private              74250 non-null int64
basin                    74250 non-null object
subvillage               73780 non-null object
region                   74250 non-null object
region_code              74250 non-null int64
district_code            74250 non-null int64
lga                      74250 non-null object
ward                     74250 non-null object
population               74250 non-null int64
public_meeting           70095 non-null object
recorded_by              74250 non-null obj

In [9]:
all_df=combined.merge(train_labels,how='outer', left_index=True, right_index=True)
all_df.tail()
#not sure that this is ok like this

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,train,status_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
74245,0.0,2013-03-04,Government Of Tanzania,1420,Government,37.581890,-3.284912,Kwa Elimisi Urio,0,Pangani,Kirueni Kati,Kilimanjaro,3,4,Moshi Rural,Mwika Kusini,1,True,GeoData Consultants Ltd,WUA,Mtiro pipeline,True,1970,gravity,gravity,gravity,wua,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,0,NaN
74246,50.0,2011-03-07,Ruthe,1428,Ruthe,35.630481,-7.710549,none,0,Rufiji,Kidete,Iringa,11,1,Iringa Rural,Kiwere,1,True,GeoData Consultants Ltd,VWC,Kiwele,True,2000,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,dry,dry,spring,spring,groundwater,communal standpipe,communal standpipe,1,non functional
74247,50.0,2013-02-16,Mission,965,DWE,35.432998,-10.639270,Kwa Mapunda,0,Ruvuma / Southern Coast,Mpakani,Ruvuma,10,2,Songea Rural,Maposeni,900,True,GeoData Consultants Ltd,VWC,Mradi wa maji wa peramiho,True,2009,other,other,other,vwc,user-group,pay per bucket,per bucket,soft,good,dry,dry,river,river/lake,surface,communal standpipe,communal standpipe,1,non functional
74248,0.0,2013-02-16,Qwickwin,1280,QWICKWIN,34.410594,-1.595024,Sarayi Mwita,0,Lake Victoria,Kebunye,Mara,20,2,Serengeti,Busawe,100,True,GeoData Consultants Ltd,WUG,QWICKWIN,True,2011,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe multiple,communal standpipe,0,NaN
74249,0.0,2013-03-11,Government Of Tanzania,1473,Central govt,37.661798,-3.608780,Kwa Mustafa Daudi,0,Pangani,Motingi,Kilimanjaro,3,2,Mwanga,Kifula,200,True,GeoData Consultants Ltd,VWC,Vulue water supply,False,1972,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,river,river/lake,surface,other,other,0,NaN


In [10]:
all_df.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,train
count,74250.000000,74250.000000,74250.000000,7.425000e+04,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000
mean,318.685704,665.667313,34.074262,-5.701771e+00,0.462330,15.265414,5.629077,180.750828,1298.463650,0.800000
std,2906.762364,692.761033,6.572519,2.944969e+00,11.537879,17.508907,9.641636,471.086120,952.349375,0.400003
min,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,33.086819,-8.525675e+00,0.000000,5.000000,2.000000,0.000000,0.000000,1.000000
50%,0.000000,364.000000,34.907475,-5.026540e+00,0.000000,12.000000,3.000000,25.000000,1986.000000,1.000000
75%,20.000000,1317.000000,37.181685,-3.325058e+00,0.000000,17.000000,5.000000,215.000000,2004.000000,1.000000
max,350000.000000,2777.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000,1.000000


In [11]:
num_set=all_df.select_dtypes(include=['float64','int64'])
num_variables=num_set.columns
num_variables
#is it ok that I have ID=index here included? I solved it, put ID into index

Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private',
       'region_code', 'district_code', 'population', 'construction_year',
       'train'],
      dtype='object')

In [12]:
all_df.describe(include=['O'])
#recorded by is a useless one, extraction type and extraction type group are redundant, same for managment and managment group, payment type and payment etc

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
count,74250,69746,69718,74250,74250,73780,74250,74250,74250,70095,74250,69404,38992,70457,74250,74250,74250,74250,74250,74250,74250,74250,74250,74250,74250,74250,74250,74250,74250,74250,59400
unique,369,2140,2410,45684,9,21425,21,125,2098,2,1,12,2868,2,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6,3
top,2011-03-17,Government Of Tanzania,DWE,none,Lake Victoria,Shuleni,Iringa,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,K,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
freq,695,11299,21751,4440,12871,646,6599,3128,386,63749,74250,45917,858,48606,33263,33263,33263,50624,65538,31712,31712,63505,63505,41522,41522,21216,21216,57286,35628,43239,32259


In [13]:
all_df['source'].unique()

array(['shallow well', 'machine dbh', 'spring', 'rainwater harvesting',
       'river', 'lake', 'other', 'dam', 'hand dtw', 'unknown'],
      dtype=object)

In [14]:
all_df['source_type'].value_counts()

spring                  21216
shallow well            21140
borehole                14930
river/lake              12914
rainwater harvesting     2863
dam                       840
other                     347
Name: source_type, dtype: int64

In [15]:
all_df['source_type'].unique()

array(['shallow well', 'borehole', 'spring', 'rainwater harvesting',
       'river/lake', 'other', 'dam'], dtype=object)

In [16]:
all_df['source_class'].unique()

array(['groundwater', 'surface', 'unknown'], dtype=object)

In [17]:
len(all_df[all_df.isnull().any(axis=1)])

46437

['date_recorded', 'funder', 'installer', 'wpt_name', 'basin',
       'subvillage', 'region', 'lga', 'ward', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group',
       'status_group']

I decided to throw out these due to too many unique:  'funder', 'installer','wpt_name','subvillage', 'ward', 'scheme_name'
I decided to throw out these due to too uniform values:  'recorded_by'   

So what remains:

In [18]:
all_df_cleaned=all_df[num_variables].copy()
len(all_df_cleaned[all_df_cleaned.isnull().any(axis=1)])

0

In [19]:
obj_variables_filtered=['date_recorded','basin',
       'region', 'lga', 'public_meeting','scheme_management', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group',
       'status_group']

#we have to do stg with date recorded here

In [20]:
all_df_cleaned=all_df[obj_variables_filtered].copy()
len(all_df_cleaned[all_df_cleaned.isnull().any(axis=1)])

24409

In [21]:
all_df.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,train,status_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,2012-11-13,Tasaf,0,TASAF,33.125828,-5.118154,Mratibu,0,Lake Tanganyika,Majengo,Tabora,14,3,Uyui,Igalula,0,NaN,GeoData Consultants Ltd,VWC,NaN,True,0,afridev,afridev,handpump,vwc,user-group,unknown,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,1,non functional
1,0.0,2011-03-05,Shipo,1978,SHIPO,34.770717,-9.395642,none,0,Rufiji,Magoda C,Iringa,11,4,Njombe,Uwemba,20,True,GeoData Consultants Ltd,NaN,NaN,False,2008,other - rope pump,rope pump,rope pump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,1,functional
2,0.0,2011-03-27,Lvia,0,LVIA,36.115056,-6.279268,Bombani,0,Wami / Ruvu,Songambele,Dodoma,1,4,Chamwino,Msamalo,0,True,GeoData Consultants Ltd,VWC,Mgun,True,0,mono,mono,motorpump,vwc,user-group,pay per bucket,per bucket,soft,good,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,1,functional
3,10.0,2013-06-03,Germany Republi,1639,CES,37.147432,-3.187555,Area 7 Namba 5,0,Pangani,Urereni,Kilimanjaro,3,5,Hai,Masama Magharibi,25,True,GeoData Consultants Ltd,Water Board,Losaa-Kia water supply,True,1999,gravity,gravity,gravity,water board,user-group,pay per bucket,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1,functional
4,0.0,2011-03-22,Cmsr,0,CMSR,36.164893,-6.099289,Ezeleda,0,Wami / Ruvu,Maata A,Dodoma,1,4,Chamwino,Majeleko,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,hand pump,hand pump,1,non functional


In [22]:
df_wdummies=pd.get_dummies(all_df, columns=['basin',
       'region', 'lga', 'public_meeting','scheme_management', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group'], drop_first=True)
df_wdummies.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,subvillage,region_code,district_code,ward,population,recorded_by,scheme_name,construction_year,train,status_group,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,basin_Rufiji,basin_Ruvuma / Southern Coast,basin_Wami / Ruvu,region_Dar es Salaam,region_Dodoma,region_Iringa,region_Kagera,region_Kigoma,region_Kilimanjaro,region_Lindi,region_Manyara,region_Mara,region_Mbeya,region_Morogoro,region_Mtwara,region_Mwanza,region_Pwani,region_Rukwa,region_Ruvuma,region_Shinyanga,region_Singida,region_Tabora,region_Tanga,lga_Arusha Urban,lga_Babati,lga_Bagamoyo,lga_Bahi,lga_Bariadi,lga_Biharamulo,lga_Bukoba Rural,lga_Bukoba Urban,lga_Bukombe,lga_Bunda,lga_Chamwino,lga_Chato,lga_Chunya,lga_Dodoma Urban,lga_Geita,lga_Hai,lga_Hanang,lga_Handeni,lga_Igunga,lga_Ilala,lga_Ileje,lga_Ilemela,lga_Iramba,lga_Iringa Rural,lga_Kahama,lga_Karagwe,lga_Karatu,lga_Kasulu,lga_Kibaha,lga_Kibondo,lga_Kigoma Rural,lga_Kigoma Urban,lga_Kilindi,lga_Kilolo,lga_Kilombero,lga_Kilosa,lga_Kilwa,lga_Kinondoni,lga_Kisarawe,lga_Kishapu,lga_Kiteto,lga_Kondoa,lga_Kongwa,lga_Korogwe,lga_Kwimba,lga_Kyela,lga_Lindi Rural,lga_Lindi Urban,lga_Liwale,lga_Longido,lga_Ludewa,lga_Lushoto,lga_Mafia,lga_Magu,lga_Makete,lga_Manyoni,lga_Masasi,lga_Maswa,lga_Mbarali,lga_Mbeya Rural,lga_Mbinga,lga_Mbozi,lga_Mbulu,lga_Meatu,lga_Meru,lga_Misenyi,lga_Missungwi,lga_Mkinga,lga_Mkuranga,lga_Monduli,lga_Morogoro Rural,lga_Morogoro Urban,lga_Moshi Rural,lga_Moshi Urban,lga_Mpanda,lga_Mpwapwa,lga_Mtwara Rural,lga_Mtwara Urban,lga_Mufindi,lga_Muheza,lga_Muleba,lga_Musoma Rural,lga_Mvomero,lga_Mwanga,lga_Nachingwea,lga_Namtumbo,lga_Nanyumbu,lga_Newala,lga_Ngara,lga_Ngorongoro,lga_Njombe,lga_Nkasi,lga_Nyamagana,lga_Nzega,lga_Pangani,lga_Rombo,lga_Rorya,lga_Ruangwa,lga_Rufiji,lga_Rungwe,lga_Same,lga_Sengerema,lga_Serengeti,lga_Shinyanga Rural,lga_Shinyanga Urban,lga_Siha,lga_Sikonge,lga_Simanjiro,lga_Singida Rural,lga_Singida Urban,lga_Songea Rural,lga_Songea Urban,lga_Sumbawanga Rural,lga_Sumbawanga Urban,lga_Tabora Urban,lga_Tandahimba,lga_Tanga,lga_Tarime,lga_Temeke,lga_Tunduru,lga_Ukerewe,lga_Ulanga,lga_Urambo,lga_Uyui,public_meeting_True,scheme_management_None,scheme_management_Other,scheme_management_Parastatal,scheme_management_Private operator,scheme_management_SWC,scheme_management_Trust,scheme_management_VWC,scheme_management_WUA,scheme_management_WUG,scheme_management_Water Board,scheme_management_Water authority,permit_True,extraction_type_cemo,extraction_type_climax,extraction_type_gravity,extraction_type_india mark ii,extraction_type_india mark iii,extraction_type_ksb,extraction_type_mono,extraction_type_nira/tanira,extraction_type_other,extraction_type_other - mkulima/shinyanga,extraction_type_other - play pump,extraction_type_other - rope pump,extraction_type_other - swn 81,extraction_type_submersible,extraction_type_swn 80,extraction_type_walimi,extraction_type_windmill,extraction_type_group_gravity,extraction_type_group_india mark ii,extraction_type_group_india mark iii,extraction_type_group_mono,extraction_type_group_nira/tanira,extraction_type_group_other,extraction_type_group_other handpump,extraction_type_group_other motorpump,extraction_type_group_rope pump,extraction_type_group_submersible,extraction_type_group_swn 80,extraction_type_group_wind-powered,extraction_type_class_handpump,extraction_type_class_motorpump,extraction_type_class_other,extraction_type_class_rope pump,extraction_type_class_submersible,extraction_type_class_wind-powered,management_other,management_other - school,management_parastatal,management_private operator,management_trust,management_unknown,management_vwc,management_water authority,management_water board,management_wua,management_wug,management_group_other,management_group_parastatal,management_group_unknown,management_group_user-group,payment_other,payment_pay annually,payment_pay monthly,payment_pay per bucket,p

In [45]:
model_variables=list(df_wdummies.columns)
model_variables.remove('date_recorded')

In [46]:
df_wdummies.groupby(['status_group'])['population'].mean()

status_group
functional                 187.553303
functional needs repair    175.102154
non functional             170.016430
Name: population, dtype: float64

In [56]:
model_variables=['amount_tsh','num_private','region_code', 'district_code', 'population', 'construction_year']

construction year matters--older the well, less likely to function-> but then you have to deal with NaNs and create bins based on date range
groupby region code? how big are the regions
, extraction type 

model_variables=['amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'region_code',
 'district_code',
 'population',
 'construction_year',
 'basin_Internal',
 'basin_Lake Nyasa',
 'basin_Lake Rukwa',
 'basin_Lake Tanganyika',
 'basin_Lake Victoria',
 'basin_Pangani',
 'basin_Rufiji',
 'basin_Ruvuma / Southern Coast',
 'basin_Wami / Ruvu',
 'region_Arusha',
 'region_Dar es Salaam',
 'region_Dodoma',
 'region_Iringa',
 'region_Kagera',
 'region_Kigoma',
 'region_Kilimanjaro',
 'region_Lindi',
 'region_Manyara',
 'region_Mara',
 'region_Mbeya',
 'region_Morogoro',
 'region_Mtwara',
 'region_Mwanza',
 'region_Pwani',
 'region_Rukwa',
 'region_Ruvuma',
 'region_Shinyanga',
 'region_Singida',
 'region_Tabora',
 'region_Tanga',
 'lga_Arusha Rural',
 'lga_Arusha Urban',
 'lga_Babati',
 'lga_Bagamoyo',
 'lga_Bahi',
 'lga_Bariadi',
 'lga_Biharamulo',
 'lga_Bukoba Rural',
 'lga_Bukoba Urban',
 'lga_Bukombe',
 'lga_Bunda',
 'lga_Chamwino',
 'lga_Chato',
 'lga_Chunya',
 'lga_Dodoma Urban',
 'lga_Geita',
 'lga_Hai',
 'lga_Hanang',
 'lga_Handeni',
 'lga_Igunga',
 'lga_Ilala',
 'lga_Ileje',
 'lga_Ilemela',
 'lga_Iramba',
 'lga_Iringa Rural',
 'lga_Kahama',
 'lga_Karagwe',
 'lga_Karatu',
 'lga_Kasulu',
 'lga_Kibaha',
 'lga_Kibondo',
 'lga_Kigoma Rural',
 'lga_Kigoma Urban',
 'lga_Kilindi',
 'lga_Kilolo',
 'lga_Kilombero',
 'lga_Kilosa',
 'lga_Kilwa',
 'lga_Kinondoni',
 'lga_Kisarawe',
 'lga_Kishapu',
 'lga_Kiteto',
 'lga_Kondoa',
 'lga_Kongwa',
 'lga_Korogwe',
 'lga_Kwimba',
 'lga_Kyela',
 'lga_Lindi Rural',
 'lga_Lindi Urban',
 'lga_Liwale',
 'lga_Longido',
 'lga_Ludewa',
 'lga_Lushoto',
 'lga_Mafia',
 'lga_Magu',
 'lga_Makete',
 'lga_Manyoni',
 'lga_Masasi',
 'lga_Maswa',
 'lga_Mbarali',
 'lga_Mbeya Rural',
 'lga_Mbinga',
 'lga_Mbozi',
 'lga_Mbulu',
 'lga_Meatu',
 'lga_Meru',
 'lga_Misenyi',
 'lga_Missungwi',
 'lga_Mkinga',
 'lga_Mkuranga',
 'lga_Monduli',
 'lga_Morogoro Rural',
 'lga_Morogoro Urban',
 'lga_Moshi Rural',
 'lga_Moshi Urban',
 'lga_Mpanda',
 'lga_Mpwapwa',
 'lga_Mtwara Rural',
 'lga_Mtwara Urban',
 'lga_Mufindi',
 'lga_Muheza',
 'lga_Muleba',
 'lga_Musoma Rural',
 'lga_Mvomero',
 'lga_Mwanga',
 'lga_Nachingwea',
 'lga_Namtumbo',
 'lga_Nanyumbu',
 'lga_Newala',
 'lga_Ngara',
 'lga_Ngorongoro',
 'lga_Njombe',
 'lga_Nkasi',
 'lga_Nyamagana',
 'lga_Nzega',
 'lga_Pangani',
 'lga_Rombo',
 'lga_Rorya',
 'lga_Ruangwa',
 'lga_Rufiji',
 'lga_Rungwe',
 'lga_Same',
 'lga_Sengerema',
 'lga_Serengeti',
 'lga_Shinyanga Rural',
 'lga_Shinyanga Urban',
 'lga_Siha',
 'lga_Sikonge',
 'lga_Simanjiro',
 'lga_Singida Rural',
 'lga_Singida Urban',
 'lga_Songea Rural',
 'lga_Songea Urban',
 'lga_Sumbawanga Rural',
 'lga_Sumbawanga Urban',
 'lga_Tabora Urban',
 'lga_Tandahimba',
 'lga_Tanga',
 'lga_Tarime',
 'lga_Temeke',
 'lga_Tunduru',
 'lga_Ukerewe',
 'lga_Ulanga',
 'lga_Urambo',
 'lga_Uyui',
 'public_meeting_False',
 'public_meeting_True',
 'scheme_management_Company',
 'scheme_management_None',
 'scheme_management_Other',
 'scheme_management_Parastatal',
 'scheme_management_Private operator',
 'scheme_management_SWC',
 'scheme_management_Trust',
 'scheme_management_VWC',
 'scheme_management_WUA',
 'scheme_management_WUG',
 'scheme_management_Water Board',
 'scheme_management_Water authority',
 'permit_False',
 'permit_True',
 'extraction_type_afridev',
 'extraction_type_cemo',
 'extraction_type_climax',
 'extraction_type_gravity',
 'extraction_type_india mark ii',
 'extraction_type_india mark iii',
 'extraction_type_ksb',
 'extraction_type_mono',
 'extraction_type_nira/tanira',
 'extraction_type_other',
 'extraction_type_other - mkulima/shinyanga',
 'extraction_type_other - play pump',
 'extraction_type_other - rope pump',
 'extraction_type_other - swn 81',
 'extraction_type_submersible',
 'extraction_type_swn 80',
 'extraction_type_walimi',
 'extraction_type_windmill',
 'extraction_type_group_afridev',
 'extraction_type_group_gravity',
 'extraction_type_group_india mark ii',
 'extraction_type_group_india mark iii',
 'extraction_type_group_mono',
 'extraction_type_group_nira/tanira',
 'extraction_type_group_other',
 'extraction_type_group_other handpump',
 'extraction_type_group_other motorpump',
 'extraction_type_group_rope pump',
 'extraction_type_group_submersible',
 'extraction_type_group_swn 80',
 'extraction_type_group_wind-powered',
 'extraction_type_class_gravity',
 'extraction_type_class_handpump',
 'extraction_type_class_motorpump',
 'extraction_type_class_other',
 'extraction_type_class_rope pump',
 'extraction_type_class_submersible',
 'extraction_type_class_wind-powered',
 'management_company',
 'management_other',
 'management_other - school',
 'management_parastatal',
 'management_private operator',
 'management_trust',
 'management_unknown',
 'management_vwc',
 'management_water authority',
 'management_water board',
 'management_wua',
 'management_wug',
 'management_group_commercial',
 'management_group_other',
 'management_group_parastatal',
 'management_group_unknown',
 'management_group_user-group',
 'payment_never pay',
 'payment_other',
 'payment_pay annually',
 'payment_pay monthly',
 'payment_pay per bucket',
 'payment_pay when scheme fails',
 'payment_unknown',
 'payment_type_annually',
 'payment_type_monthly',
 'payment_type_never pay',
 'payment_type_on failure',
 'payment_type_other',
 'payment_type_per bucket',
 'payment_type_unknown',
 'water_quality_coloured',
 'water_quality_fluoride',
 'water_quality_fluoride abandoned',
 'water_quality_milky',
 'water_quality_salty',
 'water_quality_salty abandoned',
 'water_quality_soft',
 'water_quality_unknown',
 'quality_group_colored',
 'quality_group_fluoride',
 'quality_group_good',
 'quality_group_milky',
 'quality_group_salty',
 'quality_group_unknown',
 'quantity_dry',
 'quantity_enough',
 'quantity_insufficient',
 'quantity_seasonal',
 'quantity_unknown',
 'quantity_group_dry',
 'quantity_group_enough',
 'quantity_group_insufficient',
 'quantity_group_seasonal',
 'quantity_group_unknown',
 'source_dam',
 'source_hand dtw',
 'source_lake',
 'source_machine dbh',
 'source_other',
 'source_rainwater harvesting',
 'source_river',
 'source_shallow well',
 'source_spring',
 'source_unknown',
 'source_type_borehole',
 'source_type_dam',
 'source_type_other',
 'source_type_rainwater harvesting',
 'source_type_river/lake',
 'source_type_shallow well',
 'source_type_spring',
 'source_class_groundwater',
 'source_class_surface',
 'source_class_unknown',
 'waterpoint_type_cattle trough',
 'waterpoint_type_communal standpipe',
 'waterpoint_type_communal standpipe multiple',
 'waterpoint_type_dam',
 'waterpoint_type_hand pump',
 'waterpoint_type_improved spring',
 'waterpoint_type_other',
 'waterpoint_type_group_cattle trough',
 'waterpoint_type_group_communal standpipe',
 'waterpoint_type_group_dam',
 'waterpoint_type_group_hand pump',
 'waterpoint_type_group_improved spring',
 'waterpoint_type_group_other']

In [57]:
train_df = df_wdummies[df_wdummies["train"] == 1].copy()
train_df.reset_index(level=0, inplace=True)
test_df = df_wdummies[df_wdummies["train"] == 0].copy()
test_df.reset_index(level=0, inplace=True)
#train_df.drop(["train"], axis=1, inplace=True)
#train_df.drop(['id'],axis=1, inplace=True)
#test_df.drop(["train"], axis=1, inplace=True)

In [58]:
test_df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,subvillage,region_code,district_code,ward,population,recorded_by,scheme_name,construction_year,train,status_group,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,basin_Rufiji,basin_Ruvuma / Southern Coast,basin_Wami / Ruvu,region_Dar es Salaam,region_Dodoma,region_Iringa,region_Kagera,region_Kigoma,region_Kilimanjaro,region_Lindi,region_Manyara,region_Mara,region_Mbeya,region_Morogoro,region_Mtwara,region_Mwanza,region_Pwani,region_Rukwa,region_Ruvuma,region_Shinyanga,region_Singida,region_Tabora,region_Tanga,lga_Arusha Urban,lga_Babati,lga_Bagamoyo,lga_Bahi,lga_Bariadi,lga_Biharamulo,lga_Bukoba Rural,lga_Bukoba Urban,lga_Bukombe,lga_Bunda,lga_Chamwino,lga_Chato,lga_Chunya,lga_Dodoma Urban,lga_Geita,lga_Hai,lga_Hanang,lga_Handeni,lga_Igunga,lga_Ilala,lga_Ileje,lga_Ilemela,lga_Iramba,lga_Iringa Rural,lga_Kahama,lga_Karagwe,lga_Karatu,lga_Kasulu,lga_Kibaha,lga_Kibondo,lga_Kigoma Rural,lga_Kigoma Urban,lga_Kilindi,lga_Kilolo,lga_Kilombero,lga_Kilosa,lga_Kilwa,lga_Kinondoni,lga_Kisarawe,lga_Kishapu,lga_Kiteto,lga_Kondoa,lga_Kongwa,lga_Korogwe,lga_Kwimba,lga_Kyela,lga_Lindi Rural,lga_Lindi Urban,lga_Liwale,lga_Longido,lga_Ludewa,lga_Lushoto,lga_Mafia,lga_Magu,lga_Makete,lga_Manyoni,lga_Masasi,lga_Maswa,lga_Mbarali,lga_Mbeya Rural,lga_Mbinga,lga_Mbozi,lga_Mbulu,lga_Meatu,lga_Meru,lga_Misenyi,lga_Missungwi,lga_Mkinga,lga_Mkuranga,lga_Monduli,lga_Morogoro Rural,lga_Morogoro Urban,lga_Moshi Rural,lga_Moshi Urban,lga_Mpanda,lga_Mpwapwa,lga_Mtwara Rural,lga_Mtwara Urban,lga_Mufindi,lga_Muheza,lga_Muleba,lga_Musoma Rural,lga_Mvomero,lga_Mwanga,lga_Nachingwea,lga_Namtumbo,lga_Nanyumbu,lga_Newala,lga_Ngara,lga_Ngorongoro,lga_Njombe,lga_Nkasi,lga_Nyamagana,lga_Nzega,lga_Pangani,lga_Rombo,lga_Rorya,lga_Ruangwa,lga_Rufiji,lga_Rungwe,lga_Same,lga_Sengerema,lga_Serengeti,lga_Shinyanga Rural,lga_Shinyanga Urban,lga_Siha,lga_Sikonge,lga_Simanjiro,lga_Singida Rural,lga_Singida Urban,lga_Songea Rural,lga_Songea Urban,lga_Sumbawanga Rural,lga_Sumbawanga Urban,lga_Tabora Urban,lga_Tandahimba,lga_Tanga,lga_Tarime,lga_Temeke,lga_Tunduru,lga_Ukerewe,lga_Ulanga,lga_Urambo,lga_Uyui,public_meeting_True,scheme_management_None,scheme_management_Other,scheme_management_Parastatal,scheme_management_Private operator,scheme_management_SWC,scheme_management_Trust,scheme_management_VWC,scheme_management_WUA,scheme_management_WUG,scheme_management_Water Board,scheme_management_Water authority,permit_True,extraction_type_cemo,extraction_type_climax,extraction_type_gravity,extraction_type_india mark ii,extraction_type_india mark iii,extraction_type_ksb,extraction_type_mono,extraction_type_nira/tanira,extraction_type_other,extraction_type_other - mkulima/shinyanga,extraction_type_other - play pump,extraction_type_other - rope pump,extraction_type_other - swn 81,extraction_type_submersible,extraction_type_swn 80,extraction_type_walimi,extraction_type_windmill,extraction_type_group_gravity,extraction_type_group_india mark ii,extraction_type_group_india mark iii,extraction_type_group_mono,extraction_type_group_nira/tanira,extraction_type_group_other,extraction_type_group_other handpump,extraction_type_group_other motorpump,extraction_type_group_rope pump,extraction_type_group_submersible,extraction_type_group_swn 80,extraction_type_group_wind-powered,extraction_type_class_handpump,extraction_type_class_motorpump,extraction_type_class_other,extraction_type_class_rope pump,extraction_type_class_submersible,extraction_type_class_wind-powered,management_other,management_other - school,management_parastatal,management_private operator,management_trust,management_unknown,management_vwc,management_water authority,management_water board,management_wua,management_wug,management_group_other,management_group_parastatal,management_group_unknown,management_group_user-group,payment_other,payment_pay annually,payment_pay monthly,payment_pay per bucke

In [59]:
list(test_df.columns)

['id',
 'amount_tsh',
 'date_recorded',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'wpt_name',
 'num_private',
 'subvillage',
 'region_code',
 'district_code',
 'ward',
 'population',
 'recorded_by',
 'scheme_name',
 'construction_year',
 'train',
 'status_group',
 'basin_Lake Nyasa',
 'basin_Lake Rukwa',
 'basin_Lake Tanganyika',
 'basin_Lake Victoria',
 'basin_Pangani',
 'basin_Rufiji',
 'basin_Ruvuma / Southern Coast',
 'basin_Wami / Ruvu',
 'region_Dar es Salaam',
 'region_Dodoma',
 'region_Iringa',
 'region_Kagera',
 'region_Kigoma',
 'region_Kilimanjaro',
 'region_Lindi',
 'region_Manyara',
 'region_Mara',
 'region_Mbeya',
 'region_Morogoro',
 'region_Mtwara',
 'region_Mwanza',
 'region_Pwani',
 'region_Rukwa',
 'region_Ruvuma',
 'region_Shinyanga',
 'region_Singida',
 'region_Tabora',
 'region_Tanga',
 'lga_Arusha Urban',
 'lga_Babati',
 'lga_Bagamoyo',
 'lga_Bahi',
 'lga_Bariadi',
 'lga_Biharamulo',
 'lga_Bukoba Rural',
 'lga_Bukoba Urban',
 'lga_Bukomb

# I have a problem visualising this now

import seaborn as sns
plt.figure(figsize=(15,15))
cor = merged_cases2.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [60]:
df_wdummies['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

# Model building

In [61]:
y=train_df['status_group'].copy()
X=train_df[model_variables].copy()
X.fillna(0, inplace=True)

In [62]:
#create train test split
from sklearn.model_selection import train_test_split

# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=42)

In [63]:
X.isnull().values.any()

False

In [64]:
from sklearn.preprocessing import StandardScaler as ss
sc = ss()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

C:\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Anaconda\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
classifier = SVC(kernel = 'rbf',random_state=42)
classifier.fit(X_train, y_train)

 Predicting the Test set results
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm_test = confusion_matrix(y_pred, y_test)

y_pred_train = classifier.predict(X_train)
cm_train = confusion_matrix(y_pred_train, y_train)

print()
print(cm_test)
print(cm_train)
print('Accuracy for training set for svm = {}'.format((accuracy_score(y_pred_train, y_train))))
print('Accuracy for test set for svm = {}'.format((accuracy_score(y_test, y_pred))))


In [65]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
classifier = DecisionTreeClassifier(random_state=42)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm_test = confusion_matrix(y_pred, y_test)

y_pred_train = classifier.predict(X_train)
cm_train = confusion_matrix(y_pred_train, y_train)

print()
print(cm_test)
print(cm_train)
print('Accuracy for training set for Decision Tree = {}'.format((accuracy_score(y_pred_train, y_train))))
print('Accuracy for test set for Decision Tree = {}'.format((accuracy_score(y_test, y_pred))))


[[5035  541 1969]
 [ 203  134  113]
 [1214  188 2483]]
[[23329  1545  5391]
 [   72  1459   115]
 [ 2406   450 12753]]
Accuracy for training set for Decision Tree = 0.7900042087542087
Accuracy for test set for Decision Tree = 0.6441077441077441


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
classifier = GaussianNB()
classifier.fit(X_train, y_train)


# Predicting the Test set results
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm_test = confusion_matrix(y_pred, y_test)

y_pred_train = classifier.predict(X_train)
cm_train = confusion_matrix(y_pred_train, y_train)

print()
print(cm_test)
print(cm_train)
print('Accuracy for training set for Naive Bayes = {}'.format((accuracy_score(y_pred_train, y_train))))
print('Accuracy for test set for Naive Bayes = {}'.format((accuracy_score(y_test, y_pred))))
#maybe this is not even meant for multi label classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rfc = RandomForestClassifier(criterion='entropy', n_estimators = 50,random_state=42)
rfc.fit(X_train, y_train)

# Predicting the Test set results
y_pred = rfc.predict(X_test)

from sklearn.metrics import confusion_matrix
cm_test = confusion_matrix(y_pred, y_test)

y_pred_train = rfc.predict(X_train)
cm_train = confusion_matrix(y_pred_train, y_train)

print(cm_test)
print(cm_train)
print('Accuracy for training set for Random Forest ={}'.format((accuracy_score(y_pred_train, y_train))))
#print(accuracy_score(y_test, y_pred))
print('Accuracy for test set for Random Forest = {}'.format((accuracy_score(y_test, y_pred))))

In [ ]:
rfc
#n_estimators increases runtime a bit
#'criterion': 'entropy', 'max_depth': 8, 'max_features': 'auto', 'n_estimators': 50}
#auto was: bootstrap=True, class_weight=None, criterion='gini',max_depth=None, max_features='auto', max_leaf_nodes=None
# min_impurity_decrease=0.0, min_impurity_split=None,  min_samples_leaf=1, min_samples_split=2,
#min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
#oob_score=False, random_state=42, verbose=0, warm_start=False

Initial run:

Accuracy for training set for Random Forest =0.9786826599326599

Accuracy for test set for Random Forest = 0.7908249158249159


After chaning n_est and gini to entropy:

Accuracy for training set for Random Forest =0.9943181818181818

Accuracy for test set for Random Forest = 0.7995791245791246

Classification Rate =1N∑Ni=0I(yi=yi^)
The metric used for this competition is the classification rate, which calculates the percentage of rows where the predicted class y^ in the submission matches the actual class, y in the test set. The maximum is 1 and the minimum is 0. The goal is to maximize the classification rate.

In [ ]:
param_grid = { 
    'n_estimators': [20, 50],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [ ]:
"""https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
As far as I understand now, you can specify this either via passing scoring=‘accuracy’ to the 
GridSearchCV (This is assumed to implement the scikit-learn estimator interface. 
Either estimator needs to provide a score function, or scoring must be passed.) 
OR specifying it within the Classifier. What I don't know at the moment is how to specify this 
already within the RandomForestClassifier. I know how to get the accuracy score of the model,
but I don't think this is enough here. Any thoughts?"""

from sklearn.model_selection import GridSearchCV
CV_rfc = GridSearchCV(estimator=rfc, scoring=‘accuracy’, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

In [ ]:
CV_rfc.best_params_
#{'criterion': 'entropy',
 'max_depth': 8,
 'max_features': 'auto',
 'n_estimators': 50}

rfc1=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 200, max_depth=8, criterion='gini')

In [ ]:
y_pred

# Writing predictions in the submission format

In [ ]:
test_df2=test_df[model_variables].copy()
test_df2.fillna(0, inplace=True)

In [ ]:
bekuldo_df=pd.read_csv('C:/Users/dooora/Desktop/pumpitup/SubmissionFormat.csv')
bekuldo_df['status_group']=classifier.predict(test_df2)
bekuldo_df.tail()
#test accuracy showed 0.79 for random forest, upon submission this meant 0.49, any thought why I had such a dramatic drop in performance?

In [ ]:
bekuldo_df.to_csv("submission_dori.csv",sep=',', index=False)

# Remained an open question:
can AUC be calculated for tree based models/in a classification problem? does it also apply for multilabel classification?
The issue is proposed and explained here, but no solution in this thread:
https://stats.stackexchange.com/questions/188616/how-can-we-calculate-roc-auc-for-classification-algorithm-such-as-random-forest

Here they talk about binary classification:
https://stackoverflow.com/questions/56781373/how-to-calculate-auc-for-random-forest-model-in-sklearn

It seems they have an implementation for this in R:
https://stats.stackexchange.com/questions/2151/how-to-plot-roc-curves-in-multiclass-classification

AAAnd I found its version in python:
https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html

# Ideas for improving submission score further:
carry on with data cleaning, generating new features, feature selection
test whether using the scaler improves the model or not

give XGBClassifier a try: https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn

More related materials to read https://github.com/vaibhavshukla182/Tanzania-Water-Supply/blob/master/evaluation.ipynb

https://www.kaggle.com/sociopath00/random-forest-using-gridsearchcv